In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import xgboost as xgb
from catboost import CatBoostClassifier

from sklearn.preprocessing import LabelEncoder, StandardScaler

In [24]:
url = "http://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data"
columns = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']
df = pd.read_csv(url, names=columns)
df.head()

,buying,maint,doors,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [26]:
# Convert categorical variables to numerical
df_encoded = pd.get_dummies(df, columns=['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety'])

# Features and target variable
X = df_encoded.drop('class', axis=1)
y = df_encoded['class']

# Label encoding for the target variable
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [27]:
# Train the XGBoost classifier
xgb_model = xgb.XGBClassifier(objective='multi:softmax', num_class=3)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

In [28]:
# Print out the classification report
print("XGBoost Classifier")
print(classification_report(y_test, y_pred_xgb))

XGBoost Classifier
              precision    recall  f1-score   support

           0       1.00      0.92      0.96        83
           1       0.61      1.00      0.76        11
           2       1.00      1.00      1.00       235
           3       1.00      1.00      1.00        17

    accuracy                           0.98       346
   macro avg       0.90      0.98      0.93       346
weighted avg       0.99      0.98      0.98       346



In [29]:
# Train the Catboost classifier
catboost_model = CatBoostClassifier(iterations=150, depth=5, learning_rate=0.3, loss_function='MultiClass', verbose=15)
catboost_model.fit(X_train, y_train)
y_pred_catboost = catboost_model.predict(X_test, prediction_type='Class')

0:	learn: 0.9474975	total: 143ms	remaining: 21.4s
15:	learn: 0.2026547	total: 176ms	remaining: 1.47s
30:	learn: 0.1136741	total: 209ms	remaining: 803ms
45:	learn: 0.0760837	total: 243ms	remaining: 548ms
60:	learn: 0.0548474	total: 276ms	remaining: 403ms
75:	learn: 0.0425822	total: 310ms	remaining: 302ms
90:	learn: 0.0346446	total: 346ms	remaining: 224ms
105:	learn: 0.0287692	total: 381ms	remaining: 158ms
120:	learn: 0.0247338	total: 415ms	remaining: 99.5ms
135:	learn: 0.0219709	total: 449ms	remaining: 46.2ms
149:	learn: 0.0195640	total: 482ms	remaining: 0us


In [30]:
# Print out the classification report
print("\nCatboost Classifier")
print(classification_report(y_test, y_pred_catboost))


Catboost Classifier
              precision    recall  f1-score   support

           0       1.00      0.92      0.96        83
           1       0.59      0.91      0.71        11
           2       1.00      1.00      1.00       235
           3       0.94      1.00      0.97        17

    accuracy                           0.98       346
   macro avg       0.88      0.96      0.91       346
weighted avg       0.98      0.98      0.98       346

